In [ ]:
## Here are the links to the normalized data
## weather
# 	https://s3.amazonaws.com/normalized-data-weather-bart/location2017/2/26/17/part-00000-e2cbf24e-080b-42c9-816b-abc421d2bc28.csv
# 	https://s3.amazonaws.com/normalized-data-weather-bart/main-temp2017/2/26/17/part-00000-9ad2cc92-5776-4f21-a4a4-1f0e86cf49fc.csv
# 	https://s3.amazonaws.com/normalized-data-weather-bart/weather-description2017/2/26/17/part-00000-8b0d083c-f41e-4f00-95f8-1ba26d84633a.csv
# 	https://s3.amazonaws.com/normalized-data-weather-bart/wind_df2017/2/26/17/part-00000-87d3f008-0fdd-4c3f-ae1c-c8e3dc7061ec.csv
## bart
# 	https://s3.amazonaws.com/normalized-data-weather-bart/bart-one-arrival2017/2/27/17/part-00000-7b169176-4c09-4fb4-b943-c328a64630d4.csv
# 	https://s3.amazonaws.com/normalized-data-weather-bart/bart-one-physical2017/2/27/17/part-00000-b3d0331e-0738-4e14-9196-2fc6d69575b3.csv


In [5]:
from pyspark.sql.functions import  explode, from_unixtime, from_json
from pyspark.sql import SQLContext,column
from pyspark.sql import DataFrame
import pandas as pd
import time
import pytz
import datetime

In [28]:
sqlContext = SQLContext(sc)


In [ ]:
#	https://s3.amazonaws.com/current-weather-data/2017/02/25/22/current-sf-weather-2-2017-02-25-22-40-03-80004853-a662-440b-8a99-25d76fa72d82

In [ ]:
#spark = spark session
weather_df = spark.read.json("s3a://current-weather-data/2017/02/25/22/current-sf-weather-2-2017-02-25-22-40-03-80004853-a662-440b-8a99-25d76fa72d82")

In [ ]:
weather_df .printSchema()

In [ ]:
weather_description_df = weather_df.select(from_unixtime("dt").alias('dt'),explode("weather.main"))

In [ ]:
main_temp_df = weather_df.select(from_unixtime("dt").alias('dt'),"main.humidity","main.pressure","main.temp","main.temp_max","main.temp_min")

In [ ]:
wind_df = weather_df .select(from_unixtime("dt").alias('dt'),"wind.speed", "wind.deg", "wind.gust")

In [ ]:
location_df = weather_df .select(from_unixtime("dt").alias('dt'),"name")

In [43]:
SF_time = pytz.timezone('US/Pacific')
current_sf_time = datetime.datetime.now(SF_time)
date_sf, raw_time_sf = \
        time.strftime('{}'.format(current_sf_time)).split(' ')
sf_hour, sf_minute = int(raw_time_sf[:2]), int(raw_time_sf[3:5])
now = datetime.datetime.now()
sf_year = now.year
sf_month = now.month
sf_day = now.day
KeyFileName = "{}/{}/{}/{}".format(sf_year, sf_month, sf_day, sf_hour)

In [44]:
KeyFileName

'2017/2/27/17'

In [ ]:
weather_description_df.write.save("s3a://normalized-data-weather-bart/weather-description{}".format(KeyFileName), format="csv")

In [ ]:
main_temp_df.write.save("s3a://normalized-data-weather-bart/main-temp{}".format(KeyFileName), format="csv")

In [ ]:
wind_df.write.save("s3a://normalized-data-weather-bart/wind_df{}".format(KeyFileName), format="csv")

In [ ]:
location_df.write.save("s3a://normalized-data-weather-bart/location{}".format(KeyFileName), format="csv")

In [ ]:
## onto bart data

In [7]:
bart_df = spark.read.json("s3a://bart-data-collection/2017/02/26/*/*")

In [8]:
# assume each bart train can hold 200 people
# http://www.bart.gov/about/history/cars

In [9]:
bart_df_one = bart_df.select("bike_flag.0",
               "color.0","direction.0","minutes.0","time.0","train_size.0","unix_time.0",
              "origin_station.0","destination.0","date.0")

In [10]:
pandas_bart_one = bart_df_one.toPandas()
new_bart_one_arrival = pd.DataFrame(columns=["unix_time","date","sf_time","origin_station","direction","destination"
              "minutes_til_arrival","arrival_time"])
new_bart_one_physical = pd.DataFrame(columns=["unix_time","date","sf_time","color","bike_flag","train_size","capacity"])

In [11]:
new_bart_one_physical['bike_flag']=pandas_bart_one.iloc[:,0]
new_bart_one_physical['color']=pandas_bart_one.iloc[:,1]
new_bart_one_arrival['direction']=pandas_bart_one.iloc[:,2]
new_bart_one_arrival['minutes_til_arrival']=pandas_bart_one.iloc[:,3]
new_bart_one_arrival['sf_time']=pandas_bart_one.iloc[:,4]
new_bart_one_physical['sf_time']=pandas_bart_one.iloc[:,4]
new_bart_one_physical['train_size']=pandas_bart_one.iloc[:,5]
new_bart_one_physical['capacity']=new_bart_one_physical['train_size']*200
new_bart_one_arrival['unix_time']=pandas_bart_one.iloc[:,6]
new_bart_one_physical['unix_time']=pandas_bart_one.iloc[:,6]
new_bart_one_arrival['origin_station']=pandas_bart_one.iloc[:,7]
new_bart_one_arrival['destination']=pandas_bart_one.iloc[:,8]
new_bart_one_arrival['date']=pandas_bart_one.iloc[:,9]
new_bart_one_physical['date']=pandas_bart_one.iloc[:,9]

new_bart_one_arrival['minutes_til_arrival'] = new_bart_one_arrival['minutes_til_arrival'].apply(lambda x:
                                               pd.Timedelta(x,unit='m') )


In [12]:
new_bart_one_arrival.drop('destinationminutes_til_arrival',axis=1,inplace=True)

In [13]:
new_bart_one_physical.head()

,unix_time,date,sf_time,color,bike_flag,train_size,capacity
0,1.488143e+12,02/26/2017,01:05:01 PM PST,BLUE,1,9.0,1800.0
1,1.488143e+12,02/26/2017,01:05:01 PM PST,BLUE,1,9.0,1800.0
2,1.488143e+12,02/26/2017,01:05:01 PM PST,BLUE,1,8.0,1600.0
3,1.488143e+12,02/26/2017,01:05:01 PM PST,ORANGE,1,6.0,1200.0
4,1.488143e+12,02/26/2017,01:05:01 PM PST,ORANGE,1,6.0,1200.0


In [14]:
new_bart_one_arrival.head()

,unix_time,date,sf_time,origin_station,direction,arrival_time,minutes_til_arrival,destination
0,1.488143e+12,02/26/2017,01:05:01 PM PST,Lake Merritt,South,NaN,00:15:00,Dublin/Pleasanton
1,1.488143e+12,02/26/2017,01:05:01 PM PST,24th St. Mission,North,NaN,00:14:00,Dublin/Pleasanton
2,1.488143e+12,02/26/2017,01:05:01 PM PST,24th St. Mission,South,NaN,00:13:00,Daly City
3,1.488143e+12,02/26/2017,01:05:01 PM PST,Downtown Berkeley,North,NaN,00:00:00,Richmond
4,1.488143e+12,02/26/2017,01:05:01 PM PST,Downtown Berkeley,South,NaN,00:04:00,Fremont


In [15]:
arrival_data_bart_one = []
for idx,row in enumerate(new_bart_one_arrival['sf_time']):
    
    arrival_data_bart_one.append(row+new_bart_one_arrival['minutes_til_arrival'][idx])

In [24]:
new_bart_one_arrival['arrival_time']=arrival_data_bart_one

In [25]:
new_bart_one_physical.dropna(inplace=True)
new_bart_one_arrival.dropna(inplace=True)

In [29]:
bart_one_df_final_arrival = sqlContext.createDataFrame(new_bart_one_arrival)
bart_one_df_final_physical = sqlContext.createDataFrame(new_bart_one_physical)

In [30]:
bart_one_df_final_arrival.show()

+-----------------+----------+---------------+--------------------+---------+-------------+-------------------+-------------------+
|        unix_time|      date|        sf_time|      origin_station|direction| arrival_time|minutes_til_arrival|        destination|
+-----------------+----------+---------------+--------------------+---------+-------------+-------------------+-------------------+
|1.488143115776E12|02/26/2017|01:05:01 PM PST|        Lake Merritt|    South|4801000000000|       900000000000|  Dublin/Pleasanton|
|1.488143115844E12|02/26/2017|01:05:01 PM PST|    24th St. Mission|    North|4741000000000|       840000000000|  Dublin/Pleasanton|
|1.488143115988E12|02/26/2017|01:05:01 PM PST|    24th St. Mission|    South|4681000000000|       780000000000|          Daly City|
|1.488143116056E12|02/26/2017|01:05:01 PM PST|   Downtown Berkeley|    North|3901000000000|                  0|           Richmond|
|1.488143116121E12|02/26/2017|01:05:01 PM PST|   Downtown Berkeley|    South

In [ ]:
## find the next arriving trains

In [31]:
bart_df_two = bart_df.select("bike_flag.1",
               "color.1","direction.1","minutes.1","time.1","train_size.1","unix_time.1",
              "origin_station.1","destination.1","date.1")

In [32]:
pandas_bart_two = bart_df_two.toPandas()
new_bart_two_arrival = pd.DataFrame(columns=["unix_time","date","sf_time","origin_station","direction","destination"
              "minutes_til_arrival"])
new_bart_two_physical = pd.DataFrame(columns=["unix_time","date","sf_time","color","bike_flag","train_size","capacity"])

In [33]:
new_bart_two_physical['bike_flag']=pandas_bart_two.iloc[:,0]
new_bart_two_physical['color']=pandas_bart_two.iloc[:,1]
new_bart_two_arrival['direction']=pandas_bart_two.iloc[:,2]
new_bart_two_arrival['minutes_til_arrival']=pandas_bart_two.iloc[:,3]
new_bart_two_arrival['sf_time']=pandas_bart_two.iloc[:,4]
new_bart_two_physical['sf_time']=pandas_bart_two.iloc[:,4]
new_bart_two_physical['train_size']=pandas_bart_two.iloc[:,5]
new_bart_two_physical['capacity']=new_bart_two_physical['train_size']*200
new_bart_two_arrival['unix_time']=pandas_bart_two.iloc[:,6]
new_bart_two_physical['unix_time']=pandas_bart_two.iloc[:,6]
new_bart_two_arrival['origin_station']=pandas_bart_two.iloc[:,7]
new_bart_two_arrival['destination']=pandas_bart_two.iloc[:,8]
new_bart_two_arrival['date']=pandas_bart_two.iloc[:,9]
new_bart_two_physical['date']=pandas_bart_two.iloc[:,9]

new_bart_two_arrival['minutes_til_arrival'] = new_bart_two_arrival['minutes_til_arrival'].apply(lambda x:
                                               pd.Timedelta(x,unit='m') )

In [34]:
new_bart_two_arrival.drop('destinationminutes_til_arrival',axis=1,inplace=True)

In [35]:
arrival_data_bart_two = []
for idx,row in enumerate(new_bart_two_arrival['sf_time']):
    
    arrival_data_bart_two.append(row+new_bart_two_arrival['minutes_til_arrival'][idx])
new_bart_two_arrival['arrival_time']=arrival_data_bart_two

In [ ]:
pd.to

In [36]:
new_bart_two_arrival.head()

,unix_time,date,sf_time,origin_station,direction,minutes_til_arrival,destination,arrival_time
0,1.488143e+12,02/26/2017,01:05:01 PM PST,Lake Merritt,South,00:35:00,Dublin/Pleasanton,01:40:01
1,1.488143e+12,02/26/2017,01:05:01 PM PST,24th St. Mission,North,00:34:00,Dublin/Pleasanton,01:39:01
2,1.488143e+12,02/26/2017,01:05:01 PM PST,24th St. Mission,South,00:30:00,Daly City,01:35:01
3,1.488143e+12,02/26/2017,01:05:01 PM PST,Downtown Berkeley,North,00:19:00,Richmond,01:24:01
4,1.488143e+12,02/26/2017,01:05:01 PM PST,Downtown Berkeley,South,00:24:00,Fremont,01:29:01


In [37]:
new_bart_two_physical.dropna(inplace=True)
new_bart_two_arrival.dropna(inplace=True)

In [38]:
bart_two_df_final_arrival = sqlContext.createDataFrame(new_bart_two_arrival)

In [39]:
bart_two_df_final_physical = sqlContext.createDataFrame(new_bart_two_physical)
bart_two_df_final_physical.show()

+-----------------+----------+---------------+------+---------+----------+--------+
|        unix_time|      date|        sf_time| color|bike_flag|train_size|capacity|
+-----------------+----------+---------------+------+---------+----------+--------+
|1.488143115776E12|02/26/2017|01:05:01 PM PST|  BLUE|        1|       9.0|  1800.0|
|1.488143115844E12|02/26/2017|01:05:01 PM PST|  BLUE|        1|       9.0|  1800.0|
|1.488143115988E12|02/26/2017|01:05:01 PM PST|  BLUE|        1|       8.0|  1600.0|
|1.488143116056E12|02/26/2017|01:05:01 PM PST|ORANGE|        1|       6.0|  1200.0|
|1.488143116121E12|02/26/2017|01:05:01 PM PST|ORANGE|        1|       6.0|  1200.0|
|1.488143116189E12|02/26/2017|01:05:01 PM PST|  BLUE|        1|       9.0|  1800.0|
|1.488143116256E12|02/26/2017|01:05:01 PM PST|  BLUE|        1|       8.0|  1600.0|
|1.488143116324E12|02/26/2017|01:05:01 PM PST|ORANGE|        1|       6.0|  1200.0|
|1.488143116395E12|02/26/2017|01:05:16 PM PST|ORANGE|        1|       6.0|  

In [40]:
bart_two_df_final_arrival.show()

+-----------------+----------+---------------+--------------------+---------+-------------------+-------------------+-------------+
|        unix_time|      date|        sf_time|      origin_station|direction|minutes_til_arrival|        destination| arrival_time|
+-----------------+----------+---------------+--------------------+---------+-------------------+-------------------+-------------+
|1.488143115776E12|02/26/2017|01:05:01 PM PST|        Lake Merritt|    South|      2100000000000|  Dublin/Pleasanton|6001000000000|
|1.488143115844E12|02/26/2017|01:05:01 PM PST|    24th St. Mission|    North|      2040000000000|  Dublin/Pleasanton|5941000000000|
|1.488143115988E12|02/26/2017|01:05:01 PM PST|    24th St. Mission|    South|      1800000000000|          Daly City|5701000000000|
|1.488143116056E12|02/26/2017|01:05:01 PM PST|   Downtown Berkeley|    North|      1140000000000|           Richmond|5041000000000|
|1.488143116121E12|02/26/2017|01:05:01 PM PST|   Downtown Berkeley|    South

In [45]:
bart_one_df_final_arrival.write.save("s3a://normalized-data-weather-bart/bart-one-arrival{}".format(KeyFileName),
                                     format="csv")

In [46]:
bart_one_df_final_physical.write.save("s3a://normalized-data-weather-bart/bart-one-physical{}".format(KeyFileName),
                                      format="csv")

In [47]:
bart_two_df_final_arrival.write.save("s3a://normalized-data-weather-bart/bart-two-arrival{}".format(KeyFileName),
                                     format="csv")
bart_two_df_final_physical.write.save("s3a://normalized-data-weather-bart/bart-two-physical{}".format(KeyFileName),
                                      format="csv")